In [ ]:
import os
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [ ]:
# importing data from different runs
run_name = 'Run3'

tsg_screen_data = pd.read_excel('./TSGscreen/' + run_name+'.xlsx')
tsg_screen_data.dropna(axis=0)
tsg_screen_data.dtypes
tsg_screen_data['Time'] = pd.to_datetime(tsg_screen_data['Time'],format='%H:%M:%S')
tsg_screen_data.head()

In [ ]:
eyecon_data = pd.read_excel('./Eyecondata/' + run_name+'.xlsx')
eyecon_data['Time'] = pd.to_datetime(eyecon_data['Time'],format='%H:%M:%S')
eyecon_data.head()

In [ ]:
feeder_data = pd.read_excel('./Feederdata/' + run_name+'.xlsx')
feeder_data = feeder_data.dropna(axis=1)
feeder_data['Time'] = pd.to_datetime(feeder_data['Time'],format='%H:%M:%S')
feeder_data.head()

In [ ]:
feed_eyecon = pd.merge_asof(eyecon_data,feeder_data,on='Time', tolerance=pd.Timedelta('2s'))
feed_eyecon = feed_eyecon.dropna(axis=0)
len(feed_eyecon)

In [ ]:
combined_Data = pd.merge_asof(feed_eyecon,tsg_screen_data,on='Time', tolerance=pd.Timedelta('2s'))
combined_Data = combined_Data.dropna(axis=0)
combined_Data.columns.values


In [ ]:
# plot_cols = [' D_v50', 'Torque','Zone 2','Zone 3','Zone 4','Zone 5','Zone 6','Zone 7','Zone 8']
# plot_cols = ['Zone 2','Zone 3','Zone 4','Zone 5','Zone 6','Zone 7','Zone 8']
plot_cols = [' D_v50', 'Torque']
time_arr = combined_Data.pop(combined_Data.columns.values[1])
plot_features = combined_Data[plot_cols]
plot_features.index = time_arr
_ = plot_features.plot(subplots=True)

In [ ]:
combined_Data.describe().transpose()

In [ ]:
combined_Data = combined_Data.drop([' TimeStamp','TimeStamp'],axis=1)
combined_Data = combined_Data.drop([' D_v10', ' D_v90', ' D_n10',' D_n50', ' D_n90', ' Median diameter',' Std deviation',' Shape mean', ' Shape RSD', '7 Massflow','7 Setpoint', '7 ScrewSpeed', '7 AveFeedFactor','7 DriveCommand', 'Set RPM'],axis=1)
combined_Data.dtypes


In [ ]:
column_indices = {name: i for i, name in enumerate(combined_Data.columns)}

n = len(combined_Data)
train_combined_Data = combined_Data[0:int(n*0.7)]
val_combined_Data = combined_Data[int(n*0.7):int(n*0.9)]
test_combined_Data = combined_Data[int(n*0.9):]

num_features = combined_Data.shape[1]
train_mean = train_combined_Data.mean()
train_std = train_combined_Data.std()

train_combined_Data = (train_combined_Data - train_mean) / train_std
val_combined_Data = (val_combined_Data - train_mean) / train_std
test_combined_Data = (test_combined_Data - train_mean) / train_std
combined_Data_std = (combined_Data - train_mean) / train_std
combined_Data_std = combined_Data_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=combined_Data_std)
_ = ax.set_xticklabels(combined_Data.keys(), rotation=90)



In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_combined_Data=train_combined_Data, val_combined_Data=val_combined_Data, test_combined_Data=test_combined_Data,
               label_columns=None):
    # Store the raw data.
    self.train_combined_Data = train_combined_Data
    self.val_combined_Data = val_combined_Data
    self.test_combined_Data = test_combined_Data

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_combined_Data.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

w1 = WindowGenerator(input_width=20, label_width=1, shift=1,
                     label_columns=['Torque'])
w1


In [ ]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [ ]:
example_window = tf.stack([np.array(train_combined_Data[:w1.total_window_size]),
                           np.array(train_combined_Data[50:50+w1.total_window_size]),
                           np.array(train_combined_Data[100:100+w1.total_window_size]),
                           np.array(train_combined_Data[150:150+w1.total_window_size])])

example_inputs, example_labels = w1.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

w1.example = example_inputs, example_labels

def plot(self, model=None, plot_col='Torque', max_subplots=5):
  inputs, labels = self.example
  plt.figure(figsize=(10, 6))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [seconds] / 5')

WindowGenerator.plot = plot
w1.plot()


In [ ]:
class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

baseline = Baseline(label_index=column_indices['Torque'])

baseline.compile(loss=tf.losses.MeanSquaredError(),
                 metrics=[tf.metrics.MeanAbsoluteError()])

print('Input shape:', w1.example[0].shape)
print('Output shape:', baseline(w1.example[0]).shape)

In [ ]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset
@property
def train(self):
  return self.make_dataset(self.train_combined_Data)

@property
def val(self):
  return self.make_dataset(self.val_combined_Data)

@property
def test(self):
  return self.make_dataset(self.test_combined_Data)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [ ]:
w1.train.element_spec

In [ ]:
for example_inputs, example_labels in w1.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
single_step_window = WindowGenerator(
    input_width=1, label_width=1, shift=1,
    label_columns=['Torque'])
single_step_window

combined_Data.isna().any()

In [ ]:
for example_inputs, example_labels in single_step_window.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
baseline = Baseline(label_index=column_indices['Torque'])

baseline.compile(loss=tf.losses.MeanSquaredError(),
                 metrics=[tf.metrics.MeanAbsoluteError()])

val_performance = {}
performance = {}
val_performance['Baseline'] = baseline.evaluate(single_step_window.val)
performance['Baseline'] = baseline.evaluate(single_step_window.test, verbose=0)

In [ ]:
wide_window = WindowGenerator(
    input_width=24, label_width=24, shift=1,
    label_columns=['Torque'])

wide_window
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', baseline(wide_window.example[0]).shape)
wide_window.plot(baseline)

In [ ]:
linear = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1)
])
print('Input shape:', single_step_window.example[0].shape)
print('Output shape:', linear(single_step_window.example[0]).shape)

In [ ]:
MAX_EPOCHS = 20
def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.RMSprop(learning_rate=0.0001),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

history = compile_and_fit(linear, single_step_window)

val_performance['Linear'] = linear.evaluate(single_step_window.val)
performance['Linear'] = linear.evaluate(single_step_window.test, verbose=0)

In [ ]:
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

history = compile_and_fit(dense, single_step_window)

val_performance['Dense'] = dense.evaluate(single_step_window.val)
performance['Dense'] = dense.evaluate(single_step_window.test, verbose=0)

In [ ]:
CONV_WIDTH = 3
conv_window = WindowGenerator(
    input_width=CONV_WIDTH,
    label_width=1,
    shift=1,
    label_columns=['Torque'])

conv_window

In [ ]:
conv_window.plot()
plt.title("Given 15 seconds of inputs, predict 5 seconds into the future.")

In [ ]:
multi_step_dense = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

print('Input shape:', conv_window.example[0].shape)
print('Output shape:', multi_step_dense(conv_window.example[0]).shape)

In [ ]:
history = compile_and_fit(multi_step_dense, conv_window)

val_performance['Multi step dense'] = multi_step_dense.evaluate(conv_window.val)
performance['Multi step dense'] = multi_step_dense.evaluate(conv_window.test, verbose=0)

In [ ]:
conv_window.plot(multi_step_dense)